In [1]:
import pandas as pd
import plotly.graph_objects as go
import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
import pandas as pd

#file_path = "export_result_oct21.xlsx"
file_path = "export_result_nov18.xlsx"


months = [
    "Dec23","Jan24","Feb24","Mar24","Apr24","May24",
    "Jun24","Jul24","Aug24","Sep24","Oct24","Nov24"
]

xl = pd.ExcelFile(file_path)

for sheet in months:
    df = pd.read_excel(xl, sheet_name=sheet)

    # === TABLE 1 === (A–E → columns 0:5, dynamic length)
    # Detect where table 2 starts ("food_items" in col A)
    food_start = df.index[df.iloc[:, 0].astype(str).str.lower().eq("food_items")]
    if len(food_start) > 0:
        end_t1 = food_start[0]
    else:
        end_t1 = len(df)
    table1 = df.iloc[:end_t1, 0:5].dropna(how="all")

    # === TABLE 2 === (A–D → columns 0:4, starts at "food_items")
    if len(food_start) > 0:
        start_t2 = food_start[0]
        # find next blank row to determine end
        after = df.iloc[start_t2+1:, 0].isna()
        if after.any():
            end_t2 = after.idxmax()
        else:
            end_t2 = len(df)
        table2 = df.iloc[start_t2:end_t2, 0:4].dropna(how="all")
        table2.drop(table2.index[0], inplace=True)
    else:
        table2 = pd.DataFrame()
        table2.drop(table2.index[0], inplace=True)
        
    if table2.shape[1] == 4:
        table2.columns = ['food_items', 'quantity', 'sum_of_portionunitstandard_g', 'sum_of_portionunitstandard_kg']
       
    
    # === ANIMAL & SUMMARY (H–L: cols 7–12) ===
    animal = df.iloc[0:17, 7:12].dropna(how="all")
    animal_name = ['food_items', 'quantity', 'sum_of_portionunitstandard_g',
       'gwp_of_1_kg_food', 'gwp_x_kg']
    animal.columns = animal_name
    
    summary = df.iloc[18:23, 7:12].dropna(how="all")
    summary_name = ['variables', 'mass_kg', 'percentage_kg', 'gwp_total', 'percentage_gwp']
    summary.columns = summary_name

    # === ANIMAL_S & SUMMARY_S (O–S: cols 14–19) ===
    animal_s = df.iloc[0:17, 14:19].dropna(how="all")
    animal_s.columns = animal_name
    summary_s = df.iloc[18:23, 14:19].dropna(how="all")
    summary_s.columns = summary_name

    # === DROP completely empty columns (in case of blanks)
    table1 = table1.dropna(axis=1, how="all")
    table2 = table2.dropna(axis=1, how="all")
    if table2.shape[1] == 3:
        table2.columns = ['food_items', 'sum_of_portionunitstandard_g', 'sum_of_portionunitstandard_kg']
        #print(sheet)

    # === Assign to variables dynamically ===
    suffix = sheet[:3].lower()  # e.g., dec, jan, feb
    globals()[f"{suffix}_table1"]   = table1
    globals()[f"{suffix}_table2"]   = table2
    globals()[f"{suffix}_animal"]   = animal
    globals()[f"{suffix}_summary"]  = summary
    globals()[f"{suffix}_animal_s"] = animal_s
    globals()[f"{suffix}_summary_s"]= summary_s

    #print(f"✅ Extracted all tables for {sheet}")

print("All months processed and loaded into variables.")

All months processed and loaded into variables.


In [3]:
jul_table1['gwp_per_qua'] = jul_table1['gwp_x_kg'] / jul_table1['quantity']
jul_table1

,food_items,quantity,sum_of_portionunitstandard_g,gwp_of_1_kg_food,gwp_x_kg,gwp_per_qua
0,Vegetables_MontegoBlend,7040,519480,3.44000,1787.011200,0.253837
1,A_beef_consumed_ON,15872,1427518.5,55.64676,79436.779365,5.004837
2,A_mixed_meat_consumed_ON,1067,38190,33.62227,1284.034491,1.203406
3,A_butter_consumed_ON,18636,63222.5,19.79185,1251.290237,0.067144
4,Pizza_Flatbread_Cheese_Flatbred_Tomato,293,21024,15.17000,318.934080,1.088512
...,...,...,...,...,...,...
71,CreamofWheat_Poore&Nemecek,1549,249050,0.51000,127.015500,0.081998
72,B_coffee_brewed_consumed_ON,22997,5364125,0.38919,2087.663809,0.09078
73,G_cornflakes_consumed_ON,12435,252308,0.23656,59.685980,0.0048
74,B_water_at_consumed_ON,2894,323979,0.08332,26.993930,0.009328


In [4]:
#jul_table1[['food_items','gwp_per_qua']].to_csv('sample_menu.csv', index=False)
jul_table1[['food_items','gwp_per_qua']].to_csv('sample_menu_nov22.csv', index=False)